In [110]:
import numpy as np

In [111]:
#Reading data
datafile = 'Part_A_Dataset/house-votes-84.data.txt'
cols = np.genfromtxt(datafile,dtype='str',delimiter=',')

for row in cols :
    yesCount = row[row=='y'].size
    noCount = row[row=='n'].size
    if(yesCount>=noCount):
        row[row=='?'] = 'y'
    else:
        row[row=='?'] = 'n'

#new, X, y = splitRandom(cols,25)   #to get parameters

In [112]:
def splitRandom(cols, percent):
    rng = np.random.default_rng()
    choise = np.random.choice(cols.shape[0], int(cols.shape[0] * (percent / 100)),replace=False)
    new = cols[choise]
    rem = np.delete(cols, choise, axis=0)
    # X-->inputs Y-->outputs
    X = np.array(new[:,1:])
    y = np.array(new[:,0])
    #print(new.shape,rem.shape,cols.shape)
    return new,X,y,rem

In [113]:
class Node:
    def __init__(self):
        self.col = None
        self.remainingCols = None
        self.bestIndex = None
        self.IG = None
        self.data = None
        self.EntropyYes = None
        self.EntropyNo = None
        self.rightY = None
        self.leftN = None
        self.Class = None

    def printTree(self):
        if self.leftN:
            self.leftN.printTree()
        self.printNode()
        if self.rightY:
            self.rightY.printTree()

    def printNode(self):
        print(self.col, self.remainingCols, self.bestIndex, self.IG, self.EntropyYes, self.EntropyNo, self.rightY, self.leftN, self.Class)
        if type(self.data) is np.ndarray:
            print(self.data.shape)
        else :
            print(None)

def treeSize(node):
    if node is None:
        return 0
    return 1 + treeSize(node.leftN) + treeSize(node.rightY)

In [114]:
def GetBestIGNode(data,colNames):
    y = data.T[0]
    divY = y[y=='republican'].size/y.size
    ETarget = -1*( (divY)*np.log2(divY) + (1-divY)*np.log2(1-divY) )
    IGs = []
    n = data.shape[1]
    for i in range(1,n):
        mask = np.logical_and(data.T[i]=='y', data.T[0]=='republican')
        if(data.T[i][data.T[i]=='y'].size == 0):
            div1 = 0
        else:
            div1 = data.T[i][mask].size/data.T[i][data.T[i]=='y'].size
        div2 = 1-div1
        EntropyYes = -1*((div1)*np.log2(div1) + (div2)*np.log2(div2))
        if (div1 == 0 or div1 == 1):
            EntropyYes = 0
        mask = np.logical_and(data.T[i]=='n', data.T[0]=='republican')
        if (data.T[i][data.T[i]=='n'].size == 0):
            div3 = 0
        else:
            div3 = data.T[i][mask].size/data.T[i][data.T[i]=='n'].size
        div4 = 1-div3
        EntropyNo = -1*((div3)*np.log2(div3) + (div4)*np.log2(div4))
        if (div3 == 0 or div3 == 1):
            EntropyNo = 0
        IGs.append(ETarget-((data.T[i][data.T[i]=='y'].size/data.T[i].size)*EntropyYes+(data.T[i][data.T[i]=='n'].size/data.T[i].size)*EntropyNo) )
    bestIndex = IGs.index(max(IGs))
    best = Node()
    best.col = colNames[bestIndex]
    best.remainingCols = colNames[:bestIndex]+colNames[bestIndex+1:]
    best.bestIndex = bestIndex + 1
    best.IG = IGs[bestIndex]
    best.data = data
    best.EntropyYes = EntropyYes
    best.EntropyNo = EntropyNo
    if(EntropyYes == 0):
        leaf = Node()
        leaf.Class = 'republican' if div1>div2 else 'democrat'
        best.rightY = leaf
    if(EntropyNo == 0):
        leaf = Node()
        leaf.Class = 'republican' if div3>div4 else 'democrat'
        best.leftN  = leaf
    return best

In [115]:
def buildTree(rootNode):
    #baseCase
    if(rootNode.data.shape[1] == 2):
        mask = np.logical_and(data.T[1]=='y', data.T[0]=='republican')
        if(data.T[1][data.T[1]=='y'].size == 0):
            div1 = 0
        else:
            div1 = data.T[1][mask].size/data.T[1][data.T[1]=='y'].size
        leaf = Node()
        leaf.Class = 'republican' if div1>0.5 else 'democrat'
        rootNode.rightY = leaf

        mask = np.logical_and(data.T[1]=='n', data.T[0]=='republican')
        if (data.T[1][data.T[1]=='n'].size == 0):
            div3 = 0
        else:
            div3 = data.T[1][mask].size/data.T[1][data.T[1]=='n'].size
        leaf2 = Node()
        leaf2.Class = 'republican' if div3>0.5 else 'democrat'
        rootNode.leftN = leaf2
    else:
        if(rootNode.rightY == None):
            newData = np.copy(rootNode.data)
            newData = newData[newData[:,rootNode.bestIndex] == 'y']
            newData = np.concatenate((newData[:,:rootNode.bestIndex], newData[:,rootNode.bestIndex+1:]), axis = 1)
            if newData.shape[0] == 0 :
                rootNode.Class == rootNode.data[0][0]
                return rootNode
            else:
                rootNode.rightY = buildTree(GetBestIGNode(newData,rootNode.remainingCols))
        if(rootNode.leftN == None):
            newData = np.copy(rootNode.data)
            newData = newData[newData[:,rootNode.bestIndex] == 'n']
            newData = np.concatenate((newData[:,:rootNode.bestIndex], newData[:,rootNode.bestIndex+1:]), axis = 1)
            if newData.shape[0] == 0 :
                rootNode.Class == rootNode.data[0][0]
                return rootNode
            rootNode.leftN =buildTree(GetBestIGNode(newData,rootNode.remainingCols))
    return rootNode

In [116]:
def predict(rootNode, row):
    if rootNode.Class != None:
        return rootNode.Class
    else:
        if row[rootNode.bestIndex] == 'y':
            return predict(rootNode.rightY, np.concatenate((row[:rootNode.bestIndex],row[rootNode.bestIndex+1:])))
        elif row[rootNode.bestIndex] == 'n':
            return predict(rootNode.leftN, np.concatenate((row[:rootNode.bestIndex],row[rootNode.bestIndex+1:])))

In [117]:
def predictTestData(rootNode,testData):
    right = 0
    index = 0
    for row in testData:
        right = right + (1 if predict(rootNode,row) == testData[index][0] else 0)
        index += 1
    return ((right/testData.shape[0])*100)

In [118]:
## Part_A Point_1 
## 25%rondomly and rerun 5 times and Report the sizes and accuracies of these trees in each experiment
for i in range(5):
    data, X, y, rem = splitRandom(cols,25)
    colNames = ['i1','i2','i3','i4','i5','i6','i7','i8','i9','i10','i11','i12','i13','i14','i15','i16']
    rootNode = buildTree(GetBestIGNode(data,colNames))
    print('Expermint', i+1, ':')
    print('accuracy :', predictTestData(rootNode, rem))
    print('tree size :', treeSize(rootNode), '\n')

Expermint 1 :
accuracy : 93.27217125382263
tree size : 21 

Expermint 2 :
accuracy : 92.96636085626912
tree size : 9 

Expermint 3 :
accuracy : 95.10703363914374
tree size : 23 

Expermint 4 :
accuracy : 92.66055045871559
tree size : 21 

Expermint 5 :
accuracy : 92.04892966360856
tree size : 39 



In [126]:
## Part_A Point_2 
## training set sizes (30-70%) report the mean, maximum and minimum accuracies at eachtraining set size.
## Also measure the mean, max and min tree size
sizes = [30, 40, 50, 60, 70]
accuracies = []
treeSizes = []
for size in sizes:
    data, X, y, rem = splitRandom(cols,size)
    colNames = ['i1','i2','i3','i4','i5','i6','i7','i8','i9','i10','i11','i12','i13','i14','i15','i16']
    rootNode = buildTree(GetBestIGNode(data,colNames))
    accuracies.append(predictTestData(rootNode, rem))
    treeSizes.append(treeSize(rootNode))
print('Accuracies :', accuracies)
print('Tree sizes :', treeSizes)
print('mean accuracies :', np.mean(np.array(accuracies)) )
print('maximum accuracy :', np.max(np.array(accuracies)) )
print('minimum accuracies :', np.min(np.array(accuracies)) )
print('mean sizes :', np.mean(np.array(treeSizes)) )
print('maximum sizes :', np.max(np.array(treeSizes)) )
print('minimum sizes :', np.min(np.array(treeSizes)) )

AttributeError: 'NoneType' object has no attribute 'Class'